In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import numpy as np
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical

import tensorflow as tf
import keras
import keras.backend as K

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.tensorflow_backend.set_session(sess)


LONGTERM = "/longterm/zifanw/Caltech/"
test_y = np.load(LONGTERM+'test_y.npy')
target_set_id = np.where(test_y == 55)[0]

test_x = np.load(LONGTERM+'test_x.npy')

In [ ]:
import explainer.QuantityInterests as Q
from explainer.Attribution import KerasAttr
from ruamel.yaml import YAML
from pathlib import Path
from densenet import densenet121_model
qoi = Q.ClassInterest(235)

path = Path('caltech_config.yaml')
yaml = YAML(typ='safe')
config = yaml.load(path)
img_rows, img_cols = config['img_row'], config['img_column']
img_channels = config['img_channel']
batch_size = config['batch_size']
nb_classes = config['nb_classes']

model = densenet121_model(img_rows=img_rows,
                          img_cols=img_cols,
                          color_type=img_channels,
                          num_classes=nb_classes,
                          weight_path_prefix="../")
print('=' * 50)
print('Model is created')
#     model.summary()

model.load_weights(config['weights_dir'] + config['pretrained'])




attr_creater = KerasAttr(model, qoi)

In [ ]:
X_test = np.load(config['X_test_path']).astype('float')
Y_test = np.load(config['Y_test_path'])

color_mean = [123.68, 116.779, 103.939]
for i in range(3):
    X_test[:, :, :, i] -= color_mean[i]

Y_test = to_categorical(Y_test, num_classes=nb_classes)
## Test the scores
scores = model.evaluate(X_test, Y_test, batch_size=batch_size)
print('Test loss : ', scores[0])
print('Test accuracy : ', scores[1])

In [ ]:
target_dataset = np.load(config['X_test_path']).astype('float')[target_set_id]
pred_scores = model.predict(target_dataset)
np.argmax(pred_scores, axis=-1)

In [ ]:
from explainer.Influence import KerasInflExp
import explainer as E
import explainer.Influence as Infl
import explainer.QuantityInterests as Q
import explainer.ExpertUnit as EU
import explainer.Compression as Cmpr
def largest_indices(ary, n):
    """Returns the n largest indices from a numpy array."""
    flat = ary.flatten()
    indices = np.argpartition(flat, -n)[-n:]
    indices = indices[np.argsort(-flat[indices])]
    indices = np.unravel_index(indices, ary.shape)
    x, y= indices[0], indices[1]
    xx = x.reshape((1, x.shape[0]))
    yy = y.reshape((1, y.shape[0]))
    result = [xx, yy]
    result = np.vstack(result)
    return result

def kernel(m, p=1):
    for i in range(m.shape[0]):
        for j in range(m.shape[1]):
            if m[i, j, 0] == 1:
                if i-1 >= 0:
                    m[i-1, j, :] = p
                    if j-1 >=0:
                        m[i-1, j-1, :] = p
                    if j+1<m.shape[1]:
                        m[i-1, j+1, :] = p
                if i+1 < m.shape[0]:
                    m[i+1, j, :] = p
                    if j-1 >=0:
                        m[i+1, j-1, :] = p
                    if j+1<m.shape[1]:
                        m[i+1, j+1, :] = p
                if j-1 >= 0:
                    m[i, j-1, :] = p
                if j+1 < m.shape[1]:
                    m[i, j+1, :] = p
    return m
                    

infl_model = KerasInflExp(model, channel_first=False)

In [ ]:
target = target_dataset[1:2]

In [ ]:
saliency_map = attr_creater.saliency_map(target, batch_size=16, mul_with_input=True)
integrated_grad = attr_creater.integrated_grad(target, batch_size=16, mul_with_input=True)
smooth_grad = attr_creater.smooth_grad(target, batch_size=16, mul_with_input=True)
grad_cam = attr_creater.gradcam(target, layer_name="concatenate_58", batch_size=16, channel_first=False)

In [ ]:
qoi = Q.ClassInterest(55)
raw_infl = infl_model.internal_infl(
    X_test,
    from_layer='fc6',  ## Use the pre-Softmax scores
    wrt_layer="concatenate_55",
    interest_mask=qoi)
expert = EU.Expert("Neuron")
expert(raw_infl, heuristic='max', channel_first=False)
tops = expert.get_tops()
# vis = infl_model.visualization(target,
#                                wrt_layer="concatenate_55",
#                                expert_idx=tops[:1],
#                                infl_as_wts=False,
#                                batch_size=16)
# vis = np.asarray(vis)

In [ ]:
top_idx = np.arange(10)
inter_unit_wts = expert.get_inter_unit_wts(top_idx)
multi_neuron_vis = infl_model.multi_unit_visualizaiton(target, 
                                                         "concatenate_55", 
                                                         tops[:10], 
                                                         inter_unit_wts=inter_unit_wts, 
                                                         infl_as_wts=False)

In [ ]:
vis = multi_neuron_vis

In [ ]:
# qoi = Q.ClassInterest(55)
# raw_infl = infl_model.internal_infl(
#     X_test,
#     from_layer='fc6',  ## Use the pre-Softmax scores
#     wrt_layer="relu5_11_x1",
#     interest_mask=qoi)
# channel_expert = EU.Expert("Channel")
# channel_expert(raw_infl, heuristic='max', channel_first=False)
# channel_tops = channel_expert.get_tops()
# channel_vis = infl_model.visualization(target_dataset[1:2],
#                                from_layer="fc6",
#                                wrt_layer="relu5_11_x1",
#                                expert_idx=tops[:1],
#                                infl_as_wts=True,
#                                interest_mask=qoi,
#                                batch_size=1)
# channel_vis = np.asarray(channel_vis)
channel_vis = np.load("55_Channel_relu5_11_x1_vis.npy")

In [ ]:
qoi = Q.ClassInterest(55)
raw_infl = infl_model.internal_infl(
    X_test,
    from_layer='fc6',  ## Use the pre-Softmax scores
    wrt_layer="relu5_11_x1",
    interest_mask=qoi)
channel_expert = EU.Expert("Channel")
channel_expert(raw_infl, heuristic='max', channel_first=False)
channel_tops = channel_expert.get_tops()
inter_unit_wts = channel_expert.get_inter_unit_wts(np.array([0, 1, 2]))
multi_channel_vis = infl_model.multi_unit_visualizaiton(target, 
                                                         'relu5_11_x1', 
                                                         channel_tops[:3], 
                                                         inter_unit_wts=inter_unit_wts, 
                                                         infl_as_wts=True, 
                                                         interest_mask=qoi)

# seq_of_units = channel_tops[:3]
# ave_scores = channel_expert.ave_scores
# sorted_scores = ave_scores[np.argsort(ave_scores)[::-1]][:3]
# inter_wts = sorted_scores / (np.sum(sorted_scores) + 1e-9)

# infl_score = infl_model.internal_infl(
#     target,
#     from_layer='fc6',  ## Use the pre-Softmax scores
#     wrt_layer="relu5_11_x1",
#     interest_mask=qoi)[0]

# infl_score = infl_score[: ,:, channel_tops[:3]]
# infl_score = np.maximum(infl_score, np.zeros_like(infl_score))
# intra_wts = np.transpose(infl_score / np.sum(infl_score, axis=(0, 1), keepdims=True) ,(2, 0, 1))
# intra_wts = intra_wts[None, :]
# intra_wts = np.transpose(intra_wts, (1, 0, 2, 3))
# multi_qoi = Q.MultiUnitInterest(channel_tops[:3], inter_wts, intra_wts)

# multi_channel_vis = infl_model.internal_infl(
#     target,
#     from_layer='relu5_11_x1',  ## Use the pre-Softmax scores
#     wrt_layer="input",
#     interest_mask=multi_qoi)

In [ ]:
import scipy
blur = 1
def point_cloud(grads, threshold=None):
    if len(grads.shape) == 4:
        grads = np.mean(grads, axis=-1)

    if threshold is not None:
        grads[grads < threshold] = 0

    grads /= (np.max(grads, axis=(1, 2), keepdims=True) + 1e-9)
    return grads

In [ ]:
attribution = scipy.ndimage.gaussian_filter(saliency_map[0].mean(axis=-1), sigma=5)
pos_attr_id = largest_indices(attribution, 10000)
result = []
# for i in range(pos_attr_id.shape[1]):
#     img = target[0].copy()
#     img[pos_attr_id[0][:i], pos_attr_id[1][:i], :] = 0

for i in range(0, pos_attr_id.shape[1], 10):
    mask = np.zeros((224, 224, 3))
    mask[pos_attr_id[0][:i], pos_attr_id[1][:i], :] = 1
    img = mask * target[0].copy()
    result.append(img[None, :])

removed1 = np.vstack(result)
pre_softmax_scores = infl_model.get_activation(removed1, "fc6")
print(pre_softmax_scores.shape)

In [ ]:
attribution = scipy.ndimage.gaussian_filter(integrated_grad[0].mean(axis=-1), sigma=5)
pos_attr_id = largest_indices(attribution, 10000)
result = []
# for i in range(pos_attr_id.shape[1]):
#     img = target[0].copy()
#     img[pos_attr_id[0][:i], pos_attr_id[1][:i], :] = 0
for i in range(0, pos_attr_id.shape[1], 10):
    mask = np.zeros((224, 224, 3))
    mask[pos_attr_id[0][:i], pos_attr_id[1][:i], :] = 1
    img = mask * target[0].copy()
    result.append(img[None, :])

removed2 = np.vstack(result)
pre_softmax_scores_ig = infl_model.get_activation(removed2, "fc6")

In [ ]:
attribution = scipy.ndimage.gaussian_filter(smooth_grad[0].mean(axis=-1), sigma=5)
pos_attr_id = largest_indices(attribution, 10000)

# pos_attr_id = largest_indices(smooth_grad[0].mean(axis=-1), 1000)
result = []
# for i in range(pos_attr_id.shape[1]):
#     img = target[0].copy()
#     img[pos_attr_id[0][:i], pos_attr_id[1][:i], :] = 0
for i in range(0, pos_attr_id.shape[1], 10):
    mask = np.zeros((224, 224, 3))
    mask[pos_attr_id[0][:i], pos_attr_id[1][:i], :] = 1
    img = mask * target[0].copy()
    result.append(img[None, :])

removed3 = np.vstack(result)
pre_softmax_scores_sm = infl_model.get_activation(removed3, "fc6")

In [ ]:
pos_attr_id = largest_indices(grad_cam[0].mean(axis=-1), 10000)
result = []
# for i in range(pos_attr_id.shape[1]):
#     img = target[0].copy()
#     img[pos_attr_id[0][:i], pos_attr_id[1][:i], :] = 0
for i in range(0, pos_attr_id.shape[1], 10):
    mask = np.zeros((224, 224, 3))
    mask[pos_attr_id[0][:i], pos_attr_id[1][:i], :] = 1
    img = mask * target[0].copy()
    result.append(img[None, :])

removed4 = np.vstack(result)
pre_softmax_scores_gradcam = infl_model.get_activation(removed4, "fc6")

In [ ]:
binary_map = point_cloud(vis[0] * target, threshold=0)

attribution = scipy.ndimage.gaussian_filter(binary_map[0], sigma=5)
pos_attr_id = largest_indices(attribution, 10000)


# pos_attr_id = largest_indices(binary_map[0], 1000)
result = []
# for i in range(pos_attr_id.shape[1]):
#     img = target[0].copy()
#     img[pos_attr_id[0][:i], pos_attr_id[1][:i], :] = 0
for i in range(0, pos_attr_id.shape[1], 10):
    mask = np.zeros((224, 224, 3))
    mask[pos_attr_id[0][:i], pos_attr_id[1][:i], :] = 1
    img = mask * target[0].copy()
    result.append(img[None, :])

removed5 = np.vstack(result)
pre_softmax_scores_neuron_infl = infl_model.get_activation(removed5, "fc6")

In [ ]:
binary_map = point_cloud(channel_vis[0] * target, threshold=0)
attribution = scipy.ndimage.gaussian_filter(binary_map[0], sigma=5)
pos_attr_id = largest_indices(attribution, 10000)

# pos_attr_id = largest_indices(binary_map[0], 1000)
result = []
# for i in range(pos_attr_id.shape[1]):
#     img = target[0].copy()
#     img[pos_attr_id[0][:i], pos_attr_id[1][:i], :] = 0
for i in range(0, pos_attr_id.shape[1], 10):
    mask = np.zeros((224, 224, 3))
    mask[pos_attr_id[0][:i], pos_attr_id[1][:i], :] = 1
    img = mask * target[0].copy()
    result.append(img[None, :])

removed6 = np.vstack(result)
pre_softmax_scores_channel_infl = infl_model.get_activation(removed6, "fc6")

In [ ]:
pos_attr_id = largest_indices(np.random.randn(224, 224), 10000)
result = []

# for i in range(pos_attr_id.shape[1]):
#     img = target[0].copy()
#     img[pos_attr_id[0][:i], pos_attr_id[1][:i], :] = 0
for i in range(0, pos_attr_id.shape[1], 10):
    mask = np.zeros((224, 224, 3))
    mask[pos_attr_id[0][:i], pos_attr_id[1][:i], :] = 1
    img = mask * target[0].copy()
    result.append(img[None, :])
# result = []
# img = target[0].copy()
# for i in range(1000):
#     x = np.random.randint(low=0, high=223, size=1)
#     y = np.random.randint(low=0, high=223, size=1)
#     while img[x, y, :].sum() == 0:
#         x = np.random.randint(low=0, high=223, size=1)
#         y = np.random.randint(low=0, high=223, size=1)
#     img[x, y, :] = 0
#     new_img = img.copy()
    
#     result.append(new_img[None, :])
        
removed7 = np.vstack(result)
pre_softmax_scores_random = infl_model.get_activation(removed7, "fc6")

In [ ]:
binary_map = point_cloud(multi_channel_vis * target , threshold=0)
attribution = scipy.ndimage.gaussian_filter(binary_map[0], sigma=5)
pos_attr_id = largest_indices(attribution, 10000)

# pos_attr_id = largest_indices(binary_map[0], 1000)
result = []
# for i in range(pos_attr_id.shape[1]):
#     img = target[0].copy()
#     img[pos_attr_id[0][:i], pos_attr_id[1][:i], :] = 0
for i in range(0, pos_attr_id.shape[1], 10):
    mask = np.zeros((224, 224, 3))
    mask[pos_attr_id[0][:i], pos_attr_id[1][:i], :] = 1
    img = mask * target[0].copy()
    result.append(img[None, :])

removed8 = np.vstack(result)
pre_softmax_scores_multi_channel_infl = infl_model.get_activation(removed8, "fc6")

In [ ]:
plt.figure(figsize=(12,12))
plt.plot(np.arange(len(pre_softmax_scores)), pre_softmax_scores[:, 55], label='SaliencyMap')
plt.plot(np.arange(len(pre_softmax_scores)), pre_softmax_scores_ig[:, 55], label='IG')
plt.plot(np.arange(len(pre_softmax_scores)), pre_softmax_scores_sm[:, 55], label='SM')
plt.plot(np.arange(len(pre_softmax_scores)), pre_softmax_scores_gradcam[:, 55], label='GradCAM')
plt.plot(np.arange(len(pre_softmax_scores)), pre_softmax_scores_neuron_infl[:, 55], label='NINFL')
plt.plot(np.arange(len(pre_softmax_scores)), pre_softmax_scores_channel_infl[:, 55], label='CINFL')
plt.plot(np.arange(len(pre_softmax_scores)), pre_softmax_scores_random[:, 55], label='Random')
plt.plot(np.arange(len(pre_softmax_scores)), pre_softmax_scores_multi_channel_infl[:, 55], label='M_CINFL')
plt.legend()

In [ ]:
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

layout = go.Layout(
    title='Remove pixels with high attribution',
    xaxis=dict(
        title='The number of removed pixels',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Pre-Softmax Score',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(layout=layout)

# Add traces
fig.add_trace(go.Scatter(x=np.arange(len(pre_softmax_scores)), y=pre_softmax_scores[:, 55],
                    mode='lines',
                    name='SaliencyMap'))
              
fig.add_trace(go.Scatter(x=np.arange(len(pre_softmax_scores)), y=pre_softmax_scores_ig[:, 55],
                    mode='lines',
                    name='IG'))
              
fig.add_trace(go.Scatter(x=np.arange(len(pre_softmax_scores)), y=pre_softmax_scores_sm[:, 55],
                    mode='lines',
                    name='SM'))
              
fig.add_trace(go.Scatter(x=np.arange(len(pre_softmax_scores)), y=pre_softmax_scores_gradcam[:, 55],
                    mode='lines',
                    name='GradCAM'))
              
fig.add_trace(go.Scatter(x=np.arange(len(pre_softmax_scores)), y=pre_softmax_scores_neuron_infl[:, 55],
                    mode='lines',
                    name='N_Infl'))
              
fig.add_trace(go.Scatter(x=np.arange(len(pre_softmax_scores)), y=pre_softmax_scores_channel_infl[:, 55],
                    mode='lines',
                    name='C_Infl'))
              
fig.add_trace(go.Scatter(x=np.arange(len(pre_softmax_scores)), y=pre_softmax_scores_multi_channel_infl[:, 55],
                    mode='lines',
                    name='3C_Infl'))
              
fig.add_trace(go.Scatter(x=np.arange(len(pre_softmax_scores)), y=pre_softmax_scores_random[:, 55],
                    mode='lines',
                    name='Random'))

fig.show()

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def show_imgdrop(i):
    fig, ax = plt.subplots(2,4, figsize=(15, 10))
    ax[0, 0].imshow(removed1[i].astype(int))
    ax[0, 1].imshow(removed2[i].astype(int))
    ax[0, 2].imshow(removed3[i].astype(int))
    ax[0, 3].imshow(removed4[i].astype(int))
    ax[1, 0].imshow(removed5[i].astype(int))
    ax[1, 1].imshow(removed6[i].astype(int))
    ax[1, 2].imshow(removed8[i].astype(int))
    ax[1, 3].imshow(removed7[i].astype(int))
    
    ax[0, 0].set_title('Saliency Map')
    ax[0, 1].set_title('IG')
    ax[0, 2].set_title('SM')
    ax[0, 3].set_title('GradCAM')
    ax[1, 0].set_title('Neuron Infl')
    ax[1, 1].set_title('1 Channel Infl')
    ax[1, 2].set_title('3-Channel Infl')
    ax[1, 3].set_title('Random')
    
    plt.show()
    
interact(show_imgdrop, i=widgets.IntSlider(min=0, max=999, step=1, continous_update=False))

    

In [ ]:
from explainer.Visualization import binary_mask, cv2_heatmap_mask

threhsold = 0.2
blur = 3

v_saliency_map = binary_mask(target, saliency_map, threshold=threhsold, background=0.3, blur=blur)
v_integrated_grad = binary_mask(target, integrated_grad, threshold=threhsold, background=0.3, blur=blur)
v_smooth_grad = binary_mask(target, smooth_grad, threshold=threhsold, background=0.3, blur=blur)
v_gradcam = cv2_heatmap_mask(target, grad_cam, blur=0, heat_level=1.1)
v_infl_neuron = binary_mask(target, multi_neuron_vis[0]*target, threshold=threhsold, background=0.3, blur=blur)
v_infl_channel = binary_mask(target, channel_vis[0]*target, threshold=threhsold, background=0.3, blur=blur)
v_infl_multichannel = binary_mask(target, multi_channel_vis[0]*target, threshold=threhsold, background=0.3, blur=blur)



def show_expl():
    fig, ax = plt.subplots(2,4, figsize=(15, 10))
    ax[0, 0].imshow(target[0].astype(int))
    ax[0, 1].imshow(v_saliency_map[0])
    ax[0, 2].imshow(v_integrated_grad[0])
    ax[0, 3].imshow(v_smooth_grad[0])
    ax[1, 0].imshow(v_gradcam[0])
    ax[1, 1].imshow(v_infl_neuron[0])
    ax[1, 2].imshow(v_infl_channel[0])
    ax[1, 3].imshow(v_infl_multichannel[0])
    
    ax[0, 0].set_title('Input')
    ax[0, 1].set_title('Saliency Map')
    ax[0, 2].set_title('IG')
    ax[0, 3].set_title('SM')
    ax[1, 0].set_title('GradCAM')
    ax[1, 1].set_title('Neuron Infl')
    ax[1, 2].set_title('1 Channel Infl')
    ax[1, 3].set_title('3-Channel Infl')

    
    plt.show()

show_expl()

In [ ]:
def K_Necessary(pre_softmax_scores, K=0.9, inverse=True):
    original_score = pre_softmax_scores[0]
    threhsold_score = K * original_score
    pre_softmax_scores -= threhsold_score
    for i, s in enumerate(pre_softmax_scores):
        if s < 0:
            break

    pixel_removed  = i
    
    if inverse:
        return pixel_removed / (1e-9 + (1 - K) * original_score)
    else:
        return pixel_removed
    
def BetterThanRandomK_Necessary(pre_softmax_scores, random_scores, K=0.9):
    expl = K_Necessary(pre_softmax_scores, K=K, inverse=False)
    random = K_Necessary(random_scores, K=K, inverse=False)

    return (random - expl) / random

In [ ]:
K = 0.8
print("Saliency Map: ", K_Necessary(pre_softmax_scores[:, 55].copy(), K))
print("Integrated Grad: ", K_Necessary(pre_softmax_scores_ig[:, 55].copy(), K))
print("Smooth Grad: ", K_Necessary(pre_softmax_scores_sm[:, 55].copy(), K))
print("GradCAM: ", K_Necessary(pre_softmax_scores_gradcam[:, 55].copy(), K))
print("Neuron-Infl: ", K_Necessary(pre_softmax_scores_neuron_infl[:, 55].copy(), K))
print("Channel-Infl: ", K_Necessary(pre_softmax_scores_channel_infl[:, 55].copy(), K))
print("3Channel-Infl: ", K_Necessary(pre_softmax_scores_multi_channel_infl[:, 55].copy(), K))
print("Random: ", K_Necessary(pre_softmax_scores_random[:, 55].copy(), K))



In [ ]:
BTRK_saliency_map = []
BTRK_ig = []
BTRK_sm = []
BTRK_gradcam = []
BTRK_neuron_infl = []
BTRK_channel_infl = []
BTRK_multi_channel_infl = []



for K in range(9, 0, -1):
    K *= 0.1
    BTRK_saliency_map.append(BetterThanRandomK_Necessary(pre_softmax_scores[:, 55].copy(), 
                                                         pre_softmax_scores_random[:, 55].copy(), 
                                                         K))
    BTRK_ig.append(BetterThanRandomK_Necessary(pre_softmax_scores_ig[:, 55].copy(), 
                                                     pre_softmax_scores_random[:, 55].copy(), 
                                                     K))
    BTRK_sm.append(BetterThanRandomK_Necessary(pre_softmax_scores_sm[:, 55].copy(), 
                                                     pre_softmax_scores_random[:, 55].copy(), 
                                                     K))
    BTRK_gradcam.append(BetterThanRandomK_Necessary(pre_softmax_scores_gradcam[:, 55].copy(), 
                                                         pre_softmax_scores_random[:, 55].copy(), 
                                                         K))
    BTRK_neuron_infl.append(BetterThanRandomK_Necessary(pre_softmax_scores_neuron_infl[:, 55].copy(), 
                                                     pre_softmax_scores_random[:, 55].copy(), 
                                                     K))
    BTRK_channel_infl.append(BetterThanRandomK_Necessary(pre_softmax_scores_channel_infl[:, 55].copy(), 
                                                     pre_softmax_scores_random[:, 55].copy(), 
                                                     K))
    BTRK_multi_channel_infl.append(BetterThanRandomK_Necessary(pre_softmax_scores_multi_channel_infl[:, 55].copy(), 
                                                     pre_softmax_scores_random[:, 55].copy(), 
                                                     K))

In [ ]:
fig = plt.figure(figsize=(10, 5))
plt.plot(0.1*np.arange(len(BTRK_saliency_map))[::-1], BTRK_saliency_map, label="Saliency Map")
plt.plot(0.1*np.arange(len(BTRK_saliency_map))[::-1], BTRK_ig, label="Integrated Gradients")
plt.plot(0.1*np.arange(len(BTRK_saliency_map))[::-1], BTRK_sm, label="SmoothGrad")
plt.plot(0.1*np.arange(len(BTRK_saliency_map))[::-1], BTRK_gradcam, label="GradCAM")
plt.plot(0.1*np.arange(len(BTRK_saliency_map))[::-1], BTRK_neuron_infl, label="Infl_Neuron")
plt.plot(0.1*np.arange(len(BTRK_saliency_map))[::-1], BTRK_channel_infl, label="Infl_Channel")
plt.plot(0.1*np.arange(len(BTRK_saliency_map))[::-1], BTRK_multi_channel_infl, label="Infl_Multichannel")
plt.plot(0.1*np.arange(len(BTRK_saliency_map))[::-1], [0] * len(BTRK_saliency_map), "-o", label="baseline")
plt.legend()
plt.xlabel("K value")
plt.ylabel("BTRK value")
plt.show()

In [ ]:
def point_cloud(grads, threshold=None):
    if len(grads.shape) == 4:
        grads = np.mean(grads, axis=-1)

    if threshold is not None:
        grads[grads < threshold] = 0

    grads /= (np.max(grads, axis=(1, 2), keepdims=True) + 1e-9)
    return grads

binary_map = point_cloud(integrated_grad, threshold=0)
plt.imshow(binary_map[0], cmap='gray')
plt.show()

In [ ]:
binary_map = point_cloud(smooth_grad, threshold=0)
plt.imshow(binary_map[0], cmap='gray')
plt.show()

In [ ]:
binary_map = point_cloud(multi_channel_vis * target , threshold=0)
plt.imshow(binary_map[0], cmap='gray')
plt.show()

In [ ]:
blurred = scipy.ndimage.gaussian_filter(binary_map[0], sigma=5)
plt.imshow(blurred, cmap='gray')
plt.show()